1. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.

In [1]:
import os
import requests
from datetime import datetime

def is_file_updated(file_name, new_content):
    if not os.path.exists(file_name):
        return True

    with open(file_name, 'rb') as existing_file:
        old_content = existing_file.read()

    return old_content != new_content

def download_vhi_data():
    if not os.path.exists('vhi'):
        os.mkdir('vhi')

    today = datetime.today().strftime('%Y-%m-%d')

    for ids in range(1, 26):
        file_name = f'vhi/vhi_id_{ids}_{today}.csv'
        url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"

        response = requests.get(url)
        if response.status_code == 200 and response.content.strip():
            if is_file_updated(file_name, response.content):
                with open(file_name, 'wb') as out:
                    out.write(response.content)
                print(f"Файл {file_name} оновлено ")
            else:
                print(f"Файл {file_name} вже актуальний ")
        else:
            print(f"Помилка завантаження або порожній файл для області {ids}")

if __name__ == "__main__":
    download_vhi_data()


Файл vhi/vhi_id_1_2025-03-18.csv оновлено 
Файл vhi/vhi_id_2_2025-03-18.csv оновлено 
Файл vhi/vhi_id_3_2025-03-18.csv оновлено 
Файл vhi/vhi_id_4_2025-03-18.csv оновлено 
Файл vhi/vhi_id_5_2025-03-18.csv оновлено 
Файл vhi/vhi_id_6_2025-03-18.csv оновлено 
Файл vhi/vhi_id_7_2025-03-18.csv оновлено 
Файл vhi/vhi_id_8_2025-03-18.csv оновлено 
Файл vhi/vhi_id_9_2025-03-18.csv оновлено 
Файл vhi/vhi_id_10_2025-03-18.csv оновлено 
Файл vhi/vhi_id_11_2025-03-18.csv оновлено 
Файл vhi/vhi_id_12_2025-03-18.csv оновлено 
Файл vhi/vhi_id_13_2025-03-18.csv оновлено 
Файл vhi/vhi_id_14_2025-03-18.csv оновлено 
Файл vhi/vhi_id_15_2025-03-18.csv оновлено 
Файл vhi/vhi_id_16_2025-03-18.csv оновлено 
Файл vhi/vhi_id_17_2025-03-18.csv оновлено 
Файл vhi/vhi_id_18_2025-03-18.csv оновлено 
Файл vhi/vhi_id_19_2025-03-18.csv оновлено 
Файл vhi/vhi_id_20_2025-03-18.csv оновлено 
Файл vhi/vhi_id_21_2025-03-18.csv оновлено 
Файл vhi/vhi_id_22_2025-03-18.csv оновлено 
Файл vhi/vhi_id_23_2025-03-18.csv оновлен

2. Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття. Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;
3. Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові)

In [21]:
def load_vhi_data():
    folder = 'vhi'
    files = [f for f in os.listdir(folder) if f.startswith('vhi_id_') and f.endswith('.csv')]
    df_all = []

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']

    for file_name in files:
        df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)
        df = df.drop(columns=['empty'], errors='ignore')
        df['VHI'] = pd.to_numeric(df['VHI'], errors='coerce')
        df = df[df['VHI'] != -1].dropna()

        area_id = int(file_name.split('_')[2].split('.')[0])
        df['area'] = area_id

        df_all.append(df)

    df_all = pd.concat(df_all, ignore_index=True)
    df_all = df_all.dropna(axis=1, how='all')

    df_all["Year"] = pd.to_numeric(df_all["Year"], errors="coerce")
    df_all["area"] = pd.to_numeric(df_all["area"], errors="coerce")
    df_all["VHI"] = pd.to_numeric(df_all["VHI"], errors="coerce")

    print("\nПерші 5 рядків DataFrame:")
    print(df_all.head())

    print("\nРозмір DataFrame (кількість рядків та стовпців):")
    print(df_all.shape)

    df_all = replace_area_indices(df_all)

    return df_all

def replace_area_indices(df):
    dict_areas = {
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21,
        11: 9, 12: 10, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17,
        20: 18, 21: 6, 22: 1, 23: 2, 24: 7, 25: 5
    }
    df["area"] = df["area"].replace(dict_areas)
    print("Заміна індексів NOAA на українські")
    return df

df_all = load_vhi_data()

print("Після заміни:")
print(df_all.head())



Перші 5 рядків DataFrame:
   Year  Week    SMN     SMT    VCI    TCI    VHI  area
0  1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1  1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2  1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3  1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4  1982   6.0  0.056  267.47  39.55  30.27  34.91    10

Розмір DataFrame (кількість рядків та стовпців):
(491625, 8)
Заміна індексів NOAA на українські
Після заміни:
   Year  Week    SMN     SMT    VCI    TCI    VHI  area
0  1982   2.0  0.063  261.53  55.89  38.20  47.04    21
1  1982   3.0  0.063  263.45  57.30  32.69  44.99    21
2  1982   4.0  0.061  265.10  53.96  28.62  41.29    21
3  1982   5.0  0.058  266.42  46.87  28.57  37.72    21
4  1982   6.0  0.056  267.47  39.55  30.27  34.91    21


Формується DataFrame з 491625 рядками та 8 стовпцями.
Змінюються індекси регіонів, наприклад, значення 10 стало 21.
Всі числові значення конвертуються у відповідний формат, а зайві стовпці видаляються

Ряд VHI для області за вказаний рік

In [21]:
def get_vhi_for_region_year(df, region, year):
    subset = df[(df['area'] == region) & (df['Year'] == year)]
    print(f"Ряд VHI для області {region} за {year} рік:")
    print(subset[['Week', 'VHI']])
    return subset


# Ряд VHI для області 13 (Миколаївська) за 2005 рік
get_vhi_for_region_year(df_all, region=13, year=2005)

Ряд VHI для області 13 за 2005 рік:
        Week    VHI
134431   1.0  52.27
134432   2.0  54.56
134433   3.0  57.42
134434   4.0  57.68
134435   5.0  57.74
...      ...    ...
158512  48.0  49.42
158513  49.0  47.73
158514  50.0  45.67
158515  51.0  44.60
158516  52.0  43.52

[612 rows x 2 columns]


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
134431,2005,1.0,0.106,263.45,61.40,43.13,52.27,13
134432,2005,2.0,0.098,262.00,58.26,50.86,54.56,13
134433,2005,3.0,0.088,259.85,52.02,62.83,57.42,13
134434,2005,4.0,0.077,258.07,42.31,73.05,57.68,13
134435,2005,5.0,0.069,256.66,33.14,82.34,57.74,13
...,...,...,...,...,...,...,...,...
158512,2005,48.0,0.154,270.64,63.91,34.93,49.42,13
158513,2005,49.0,0.126,267.49,53.73,41.72,47.73,13
158514,2005,50.0,0.106,265.09,47.31,44.04,45.67,13
158515,2005,51.0,0.093,263.78,44.77,44.42,44.60,13


Дані показують індекс VHI (Vegetation Health Index) для кожного тижня 2005 року.
На початку року (1-й тиждень) VHI = 52.27, що свідчить про середній рівень рослинного покриву.
До 3-5 тижня VHI зростає (до 57.74), що може свідчити про покращення стану рослинності.
Восени (48-52 тиждень) VHI знижується (до 43.52), що може бути пов’язано з сезонними змінами

Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани

In [22]:
def get_vhi_extremes(df, regions, years):
    subset = df[df['area'].isin(regions) & df['Year'].isin(years)]

    min_vhi = subset['VHI'].min()
    max_vhi = subset['VHI'].max()
    mean_vhi = subset['VHI'].mean()
    median_vhi = subset['VHI'].median()

    print(f" Статистика VHI для областей {regions} за {years}:")
    print(f"   ➜ Мінімум: {min_vhi}")
    print(f"   ➜ Максимум: {max_vhi}")
    print(f"   ➜ Середнє: {mean_vhi:.2f}")
    print(f"   ➜ Медіана: {median_vhi:.2f}")

    return min_vhi, max_vhi, mean_vhi, median_vhi

get_vhi_extremes(df_all, regions=[13, 14], years=[2005, 2010])

 Статистика VHI для областей [13, 14] за [2005, 2010]:
   ➜ Мінімум: 30.57
   ➜ Максимум: 74.85
   ➜ Середнє: 50.39
   ➜ Медіана: 51.29


(30.57, 74.85, 50.38966019417475, 51.29)

Мінімальне значення VHI: 30.57
У певні тижні в Миколаївській або Одеській області рослинність була в поганому стані.

Максимальне значення VHI: 74.85
У деякі періоди індекс досягав дуже хороших умов для рослинності.

Середнє значення VHI: 50.39
В загальному показник був середнім, без критичних відхилень.

Медіана VHI: 51.29
Половина значень була нижче цього рівня, половина вище.

Ряд VHI за вказаний діапазон років для вказаних областей

In [27]:
def get_vhi_for_year_range(df, regions, start_year, end_year):
    subset = df[(df['area'].isin(regions)) & (df['Year'].between(start_year, end_year))]
    print(f"Ряд VHI для областей {regions} за {start_year}-{end_year} роки:")
    print(subset[['Year', 'Week', 'area', 'VHI']])
    return subset

get_vhi_for_year_range(df_all, regions=[13, 14], start_year=2000, end_year=2010)

Ряд VHI для областей [13, 14] за 2000-2010 роки:
        Year  Week  area    VHI
134190  2000   1.0    13  38.25
134191  2000   2.0    13  40.00
134192  2000   3.0    13  41.05
134193  2000   4.0    13  41.56
134194  2000   5.0    13  43.12
...      ...   ...   ...    ...
184992  2010  48.0    14  37.29
184993  2010  49.0    14  38.57
184994  2010  50.0    14  38.90
184995  2010  51.0    14  37.16
184996  2010  52.0    14  35.24

[13248 rows x 4 columns]


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
134190,2000,1.0,0.049,259.87,18.39,58.11,38.25,13
134191,2000,2.0,0.048,259.79,19.35,60.66,40.00,13
134192,2000,3.0,0.051,260.83,23.05,59.05,41.05,13
134193,2000,4.0,0.057,262.38,27.72,55.41,41.56,13
134194,2000,5.0,0.067,263.97,31.59,54.65,43.12,13
...,...,...,...,...,...,...,...,...
184992,2010,48.0,0.128,275.88,57.68,16.90,37.29,14
184993,2010,49.0,0.108,272.54,53.07,24.08,38.57,14
184994,2010,50.0,0.091,269.51,48.60,29.20,38.90,14
184995,2010,51.0,0.072,266.97,40.27,34.06,37.16,14


Перші рядки (Year = 2000, area = 13):
VHI зростає від 38.25 до 43.12 (стабільне покращення)

Останні рядки (Year = 2010, area = 14):
VHI знижується від 38.90 до 35.24 (можливо, погіршення стану рослинності)

Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25).

In [35]:
def find_drought_years(df, threshold_percentage=20):
    total_regions = df['area'].nunique()
    threshold = max(5, int((threshold_percentage / 100) * total_regions))

    drought_data = {}

    for year in sorted(df['Year'].unique()):
        yearly_data = df[df['Year'] == year]
        affected_regions = yearly_data[yearly_data['VHI'] < 15][['area', 'VHI']]

        if affected_regions['area'].nunique() >= threshold:
            drought_data[year] = affected_regions.groupby('area')['VHI'].min().reset_index()

    if not drought_data:
        print("Екстремальних посух не виявлено.")
        return

    print("\n **Роки з екстремальними посухами (VHI < 15)** \n")
    print(f"{'Рік':<6} {'Область':<10} {'VHI':<6}")
    print("-" * 26)

    for year, df_regions in drought_data.items():
        for _, row in df_regions.iterrows():
            print(f"{year:<6} {row['area']:<10} {row['VHI']:<6.2f}")

find_drought_years(df_all, threshold_percentage=20)


 **Роки з екстремальними посухами (VHI < 15)** 

Рік    Область    VHI   
--------------------------
2000   7.0        11.25 
2000   9.0        10.60 
2000   10.0       6.49  
2000   18.0       8.14  
2000   19.0       9.36  
2000   22.0       10.68 


2000 рік був екстремально посушливим для 6 областей
Найгірша ситуація в області 10.0 (VHI = 6.49, дуже сильна посуха)
Інші області мали VHI між 8.14 і 11.25, що також вказує на значну посуху